# CAPSTONE INVESTIGACIÓN OPERATIVA

### Instalación de dependencias en el .venv

In [15]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install numpy pandas scikit-learn matplotlib joblib gurobipy gurobi-machinelearning

  Using cached gurobi_machinelearning-1.5.5-py3-none-any.whl.metadata (6.8 kB)
Using cached gurobi_machinelearning-1.5.5-py3-none-any.whl (73 kB)


### Importamos Librerias necesarias

In [1]:
# ============
# Config & IO
# ============
import re, numpy as np, pandas as pd
from pathlib import Path

from sklearn.model_selection import KFold, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
# === PASO 3.2 — Fallback SIN Pipeline (estimador puro) ===
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Registramos convertidores sklearn por si acaso (aunque no usamos pipeline)
import gurobi_ml, gurobi_ml.sklearn
from gurobi_ml import add_predictor_constr

from sklearn.tree import DecisionTreeRegressor
# ===============================
# Comparación de modelos (CV 5-fold)
# ===============================
import time, numpy as np, pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV


### Prepocesamiento de la base de datos "AmesHousing"

In [7]:
# Ajusta la ruta a tu CSV Ames
FILE = "AmesHousing(in).csv"   # cámbialo si está en otra carpeta

def clean_col(c: str) -> str:
    c = re.sub(r"[^0-9A-Za-z_]+", "_", c)
    c = re.sub(r"_+", "_", c).strip("_")
    return c

df = pd.read_csv(FILE)
orig_cols = df.columns.tolist()
df.columns = [clean_col(c) for c in df.columns]
colmap = dict(zip(orig_cols, df.columns))

#Arreglar variable Fence si es Na = 0, sino 1
df["Fence"] = df["Fence"].apply(lambda x: 0 if pd.isna(x) else 1)


# Detectar objetivo (SalePrice o similar)
target_candidates = [c for c in df.columns if re.fullmatch(r"(?i)sale_?price", c)]
assert target_candidates, "No encontré la columna objetivo (SalePrice)"
TARGET = target_candidates[0]

# Quitar columnas problemáticas/ID (ajusta si quieres)
ID_COLS = [c for c in ["Order","PID","Pool_QC","Misc_Feature","Alley","Fireplace_Qu"] if c in df.columns]

X = df.drop(columns=[TARGET] + ID_COLS, errors="ignore").copy()
y = df[TARGET].copy()

# Detectar tipos
num_cols = [c for c in X.columns if np.issubdtype(X[c].dtype, np.number)]
cat_cols = [c for c in X.columns if c not in num_cols]

def make_ohe():
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

num_pre = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pre = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("ohe", make_ohe())
])

pre = ColumnTransformer([
    ("num", num_pre, num_cols),
    ("cat", cat_pre, cat_cols)
])

print(f"Shape: {df.shape} | TARGET: {TARGET}")
print("Numéricas:", len(num_cols), "| Categóricas:", len(cat_cols))


Shape: (2930, 82) | TARGET: SalePrice
Numéricas: 37 | Categóricas: 38


### Modelos  ML

In [ ]:

CANDIDATES = [
    "Gr_Liv_Area","Total_Bsmt_SF","Garage_Area",
    "Garage_Cars","Full_Bath","Fireplaces","Overall_Qual", 
]
assert 'X' in globals() and 'y' in globals(), "Debes tener X,y en memoria."
X_fit = X[CANDIDATES].copy()
X_fit = X_fit.fillna(X_fit.median(numeric_only=True))
y_log = np.log1p(y)  # entrenamos en log1p

# ---------- Modelos a probar ----------
models = {
    "Tree_d10": DecisionTreeRegressor(max_depth=10, min_samples_leaf=5, random_state=42),
    "Tree_d14": DecisionTreeRegressor(max_depth=14, min_samples_leaf=3, random_state=42),

    "RF_200": RandomForestRegressor(
        n_estimators=200, max_depth=16, min_samples_leaf=3, max_features="sqrt",
        n_jobs=-1, random_state=42
    ),
    "RF_400": RandomForestRegressor(
        n_estimators=400, max_depth=18, min_samples_leaf=2, max_features="sqrt",
        n_jobs=-1, random_state=42
    ),

    "GBR": GradientBoostingRegressor(
        learning_rate=0.05, n_estimators=400, max_depth=3, subsample=0.9, random_state=42
    ),

    "Linear": LinearRegression(),
    "RidgeCV": RidgeCV(alphas=np.logspace(-3,3,7)),
    "LassoCV": LassoCV(cv=5, random_state=42, max_iter=5000)
}

def cv_eval_estimator(est, X, y_log, n_splits=5, seed=42):
    """Entrena en log1p(y) y reporta métricas en escala original."""
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    y_true_all, y_pred_all = [], []
    t0 = time.time()
    for tr, te in kf.split(X):
        Xt, Xv = X.iloc[tr], X.iloc[te]
        if hasattr(y_log, "iloc"):
            yt, yv = y_log.iloc[tr], y_log.iloc[te]
        else:
            yt, yv = y_log[tr], y_log[te]

        m = est.__class__(**est.get_params())
        m.fit(Xt, yt)
        pred_v = np.expm1(m.predict(Xv))  # back to original scale
        yv_o   = np.expm1(yv)
        y_true_all.append(yv_o); y_pred_all.append(pred_v)
    elapsed = time.time() - t0

    y_true = np.concatenate(y_true_all); y_pred = np.concatenate(y_pred_all)
    mse  = mean_squared_error(y_true, y_pred)   # <-- sin 'squared'
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    return {"RMSE": rmse, "MAE": mae, "R2": r2, "secs": elapsed}

# ---------- Evaluación ----------
rows = []
for name, est in models.items():
    metrics = cv_eval_estimator(est, X_fit, y_log, n_splits=5, seed=42)
    rows.append({"model": name, **{k: round(v,3) if k!='secs' else round(v,2) for k,v in metrics.items()}})
results = pd.DataFrame(rows).sort_values("RMSE")
display(results)

best_name = results.iloc[0]["model"]
print(f"\nMejor por RMSE (CV): {best_name}")

# ---------- Entrenar todos en TODO el dataset para usarlos luego ----------
fitted_models = {}
for name, est in models.items():
    m = est.__class__(**est.get_params())
    m.fit(X_fit, y_log)
    fitted_models[name] = m

feat_names = list(X_fit.columns)
print("Features usadas:", feat_names)

# ---------- Importancias / Coefs del ganador ----------
winner = fitted_models[best_name]
if hasattr(winner, "feature_importances_"):
    imp = pd.Series(winner.feature_importances_, index=feat_names).sort_values(ascending=False)
    print("\nImportancias del ganador:"); display(imp)
elif hasattr(winner, "coef_"):
    coefs = pd.Series(np.ravel(winner.coef_), index=feat_names).sort_values(key=np.abs, ascending=False)
    print("\nCoeficientes (ordenados por |coef|) del ganador:"); display(coefs)
else:
    print("\nEl modelo ganador no expone importancias/coefs directamente.")

# ---------- Recomendado para Gurobi ----------
embeedable = {"Tree_d10","Tree_d14","RF_200","RF_400","GBR","Linear","RidgeCV"}
df_emb = results[results["model"].isin(embeedable)].sort_values("RMSE")
best_emb_name = df_emb.iloc[0]["model"]
best_emb_model = fitted_models[best_emb_name]
print(f"\n→ Recomendado para Gurobi (soporte y performance): {best_emb_name}")


,model,RMSE,MAE,R2,secs
4,GBR,28747.239,19353.107,0.870,6.80
3,RF_400,29995.829,19433.458,0.859,4.94
2,RF_200,30416.159,19582.617,0.855,2.47
0,Tree_d10,34650.419,22907.941,0.812,0.13
1,Tree_d14,36144.353,24278.598,0.795,0.10
5,Linear,46632.372,21770.416,0.659,0.02
6,RidgeCV,46866.869,21757.656,0.656,0.04
7,LassoCV,113788.555,30513.773,-1.030,0.30



Mejor por RMSE (CV): GBR
Features usadas: ['Gr_Liv_Area', 'Total_Bsmt_SF', 'Garage_Area', 'Garage_Cars', 'Full_Bath', 'Fireplaces', 'Overall_Qual', 'Fence']

Importancias del ganador:


Overall_Qual     0.590492
Gr_Liv_Area      0.165508
Total_Bsmt_SF    0.107026
Garage_Cars      0.050787
Garage_Area      0.045887
Fireplaces       0.029425
Full_Bath        0.010093
Fence            0.000783
dtype: float64


→ Recomendado para Gurobi (soporte y performance): GBR


### Parámetros del modelo

In [ ]:
# ----------------------------
# 2) Definir baseline, límites y costos
# ----------------------------
ID_CASA = 2100

# baseline con las MISMAS columnas usadas para entrenar el predictor
baseline = X_fit.iloc[ID_CASA].astype(float)

print("Caso base:")
print(baseline)
predictor = best_emb_model
# predicción del modelo (viene en log1p -> deshacer con expm1)
pred_log = predictor.predict(baseline.to_frame().T)[0]
precio_base_pred = float(np.expm1(pred_log))
print(f"Precio base predicho: {precio_base_pred:,.0f}")

# precio real en escala original (NO aplicar expm1)
precio_real = float(y.iloc[ID_CASA])  # ojo: iloc
print(f"Precio real: {precio_real:,.0f}")

# percentil 95 para cada feature (usado luego para cotas superiores)
quant95 = X_fit.quantile(0.95)


default_costs = {
    "Gr_Liv_Area":   200,     # USD por ft²
    "Total_Bsmt_SF":  80,     # USD por ft²
    "Garage_Area":    60,     # USD por ft²
    "Garage_Cars":  17000,    # USD por puesto
    "Full_Bath":    25000,    # USD por baño
    "Fireplaces":    6000,    # USD por chimenea
    "Overall_Qual": 20000    # USD por subir 1 punto (proxy)
}

room = {
    "Gr_Liv_Area":   400.0,
    "Total_Bsmt_SF": 300.0,
    "Garage_Area":   250.0,
    "Garage_Cars":     1.0,   # si tu lote permite 2, súbelo a 2.0
    "Full_Bath":       1.0,
    "Fireplaces":      1.0,
    "Overall_Qual":    1.0    # 1 punto suele ser más realista que 2
}

bounds, costs = {}, {}
for c in feat_names:
    base = float(baseline.get(c, X_fit[c].median()))
    ub_by_room = base + room.get(c, 0.0)
    ub_by_p95  = float(quant95[c])
    lb = base
    ub = max(lb, min(ub_by_room, ub_by_p95))
    bounds[c] = (lb, ub)
    costs[c]  = default_costs.get(c, 0.0)

BUDGET = 200_000

Caso base:
Gr_Liv_Area      1494.0
Total_Bsmt_SF    1494.0
Garage_Area       840.0
Garage_Cars         3.0
Full_Bath           2.0
Fireplaces          1.0
Overall_Qual        7.0
Fence               0.0
Name: 2100, dtype: float64
Precio base predicho: 232,757
Precio real: 279,500


### Guroby

In [10]:

# ---------- Parámetros ----------
MODEL_NAME     = "auto"          # "auto" o uno de: "RF_400","GBR","Tree_d14","Linear","RidgeCV", etc.
OBJECTIVE_MODE = "profit"        # "price" | "profit" | "roi"
ROI_MIN        = 0.10            # usado sólo si OBJECTIVE_MODE == "roi"
PWL_K          = 25              # segmentos para PWL (log1p -> price)

# ---------- Resolver el estimador ----------
assert 'fitted_models' in globals(), "Falta 'fitted_models' (corre la celda de comparación)."
compatibles = {"Tree_d10","Tree_d14","RF_200","RF_400","GBR","Linear","RidgeCV"}

if MODEL_NAME == "auto":
    if 'best_emb_name' in globals():
        chosen = best_emb_name
    elif 'best_name' in globals():
        if best_name in compatibles:
            chosen = best_name
        else:
            assert 'results' in globals(), "Falta 'results' para elegir el mejor compatible."
            chosen = results[results["model"].isin(compatibles)].sort_values("RMSE").iloc[0]["model"]
    else:
        chosen = next((k for k in fitted_models if k in compatibles), None)
        assert chosen is not None, "No hay modelo compatible en 'fitted_models'."
else:
    chosen = MODEL_NAME
    assert chosen in fitted_models, f"'{chosen}' no está en fitted_models."

predictor = fitted_models[chosen]
print(f"→ Usando modelo: {chosen}")

# ---------- Columnas esperadas ----------
if hasattr(predictor, "feature_names_in_"):
    feat_names_model = list(predictor.feature_names_in_)
else:
    assert 'feat_names' in globals(), "Falta 'feat_names'."
    feat_names_model = list(feat_names)

for need in ("baseline","bounds","costs","BUDGET"):
    assert need in globals(), f"Falta '{need}' en el entorno."
missing = [c for c in feat_names_model if c not in bounds or c not in baseline.index or c not in costs]
assert not missing, f"Faltan claves en bounds/baseline/costs para: {missing}"

# ---------- Modelo de optimización ----------
m = gp.Model(f"ames_renov_opt_{chosen}")

int_like = {"Garage_Cars","Full_Bath","Fireplaces","Overall_Qual"}
x = {}
for c in feat_names_model:
    lb, ub = bounds[c]
    if c in int_like:
        lb_i, ub_i = int(np.floor(lb)), int(np.ceil(ub))
        if ub_i < lb_i: ub_i = lb_i
        x[c] = m.addVar(lb=lb_i, ub=ub_i, vtype=GRB.INTEGER,   name=c)
    else:
        x[c] = m.addVar(lb=float(lb), ub=float(ub), vtype=GRB.CONTINUOUS, name=c)

# Presupuesto (sólo mejoras)
cost_expr = gp.quicksum(costs[c] * (x[c] - float(baseline[c])) for c in feat_names_model)
m.addConstr(cost_expr <= float(BUDGET), name="Budget")

# (Opcional) ejemplo garaje acoplado:
# STALL_SF = 200.0
# if "Garage_Area" in x and "Garage_Cars" in x:
#     m.addConstr((x["Garage_Area"] - float(baseline["Garage_Area"])) >=
#                 STALL_SF * (x["Garage_Cars"] - float(baseline["Garage_Cars"])),
#                 name="GarageAreaCoversStalls")

# DF 1xN con el orden exacto de columnas
x_df = pd.DataFrame([[x[c] for c in feat_names_model]], columns=feat_names_model)

# Variable de salida del predictor (log1p del precio)
y_pred_log = m.addVar(name="y_pred_log")

add_predictor_constr(
    gp_model=m,
    predictor=predictor,
    input_vars=x_df,
    output_vars=y_pred_log
)

# ---------- Objetivo ----------
if OBJECTIVE_MODE == "price":
    # Maximiza log1p(price): suficiente porque exp() es monótona
    m.setObjective(y_pred_log, GRB.MAXIMIZE)

else:
    # Necesitamos precio en escala original: price ≈ exp(y_log) - 1 vía PWL
    # Rango PWL desde tus datos (más robusto que fijo)
    y_log_all = np.log1p(y) if hasattr(y, "__len__") else pd.Series([0.0])
    ymin = float(np.percentile(y_log_all, 1))
    ymax = float(np.percentile(y_log_all, 99))
    if not np.isfinite(ymin) or not np.isfinite(ymax) or ymin >= ymax:
        ymin, ymax = 10.5, 13.5  # fallback razonable

    xs = np.linspace(ymin, ymax, int(PWL_K)).tolist()
    ys = (np.expm1(xs)).tolist()

    price = m.addVar(name="price")                      # USD (o tu moneda de y)
    m.addGenConstrPWL(y_pred_log, price, xs, ys, name="log_to_price")

    # Precio baseline (constante, mismo predictor)
    baseline_vec = pd.DataFrame([baseline[feat_names_model].to_dict()], columns=feat_names_model)
    price_before = float(np.expm1(predictor.predict(baseline_vec))[0])

    if OBJECTIVE_MODE == "profit":
        # Max (precio_after - costo)
        m.setObjective(price - cost_expr, GRB.MAXIMIZE)
    elif OBJECTIVE_MODE == "roi":
        # Max (precio_after - costo) con ROI mínimo
        m.addConstr(price - price_before >= ROI_MIN * cost_expr, name="ROImin")
        m.setObjective(price - cost_expr, GRB.MAXIMIZE)
    else:
        raise ValueError("OBJECTIVE_MODE debe ser 'price', 'profit' o 'roi'.")

# ---------- Resolver ----------
m.Params.OutputFlag = 1
m.optimize()

# ---------- Reporte ----------
if m.SolCount > 0:
    opt_vec = pd.DataFrame([{c: x[c].X for c in feat_names_model}], columns=feat_names_model)

    price_before_rep = float(np.expm1(predictor.predict(baseline_vec))[0])
    if OBJECTIVE_MODE == "price":
        price_after_rep = float(np.expm1(y_pred_log.X))  # aprox (sin PWL)
    else:
        price_after_rep = float(price.X)                 # con PWL

    deltas = {c: x[c].X - float(baseline[c]) for c in feat_names_model}
    spent  = float(cost_expr.getValue())
    profit = price_after_rep - price_before_rep - spent
    roi    = (profit / spent) if spent > 0 else float('nan')

    print("\n=== SOLUCIÓN ===")
    print(f"Modelo usado    : {chosen}")
    print(f"Objetivo        : {OBJECTIVE_MODE}")
    print(f"Precio antes    : {price_before_rep:,.0f}")
    print(f"Precio después  : {price_after_rep:,.0f}")
    print(f"Gasto estimado  : {spent:,.0f}  (<= presupuesto {BUDGET:,.0f})")
    print(f"Ganancia        : {profit:,.0f}   |  ROI: {roi:,.2f}")

    print("\nValores finales de features:")
    for c in feat_names_model:
        print(f" - {c:15s}: {x[c].X:10.3f}  (base {float(baseline[c]):10.3f}, Δ={deltas[c]:+10.3f})")
else:
    print("No se encontró solución factible.")


→ Usando modelo: GBR
Set parameter Username
Set parameter LicenseID to value 2670211
Academic license - for non-commercial use only - expires 2026-05-23
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1202 rows, 3518 columns and 4316 nonzeros
Model fingerprint: 0x2a3e757b
Model has 779 simple general constraints
  778 INDICATOR, 1 PWL
Variable types: 406 continuous, 3112 integer (3108 binary)
Coefficient statistics:
  Matrix range     [5e-02, 3e+04]
  Objective range  [1e+00, 3e+04]
  Bounds range     [1e+00, 2e+03]
  RHS range        [9e-03, 9e+05]
  PWLCon x range   [1e+01, 1e+01]
  PWLCon y range   [7e+04, 4e+05]
  GenCon rhs range [1e-05, 2e+03]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 897 rows and 3321 colum